<div style="border:solid green 4px; padding: 20px">Привет! Мои критичные комментарии ты сможешь найти в <span style='color: red;'>красных</span> блоках, замечания в <span style='color: #ebd731;'>жёлтых</span>, а рекомендации и прочую информацию - в <span style='color: green;'>зеленых</span>.</div>

# Определение перспективного тарифа для телеком компании

Перед нами данные компании «Мегалайн» — федерального оператора сотовой связи. Клиентам предлагают два тарифных плана: **«Смарт»** и **«Ультра»**. Чтобы скорректировать рекламный бюджет, коммерческий департамент хочет понять, какой тариф приносит больше денег.

Нам предстоит сделать предварительный анализ тарифов на небольшой выборке клиентов. В нашем распоряжении данные 500 пользователей «Мегалайна»: кто они, откуда, каким тарифом пользуются, сколько звонков и сообщений каждый отправил за 2018 год. 

Нужно проанализировать поведение клиентов и сделать вывод — какой тариф лучше.

## Шаг 1. Откройте файл с данными и изучите общую информацию
    

Подключим сначала все необходимые баблиотеки, которые нам потребуются в ходе выполнения проекта.

In [1]:
import pandas as pd

import matplotlib
from matplotlib import pyplot as plt
import plotly.graph_objects as go

from math import factorial
from scipy import stats as st
import numpy as np

In [2]:
calls = pd.read_csv('/datasets/calls.csv')
internet = pd.read_csv('/datasets/internet.csv')
messages= pd.read_csv('/datasets/messages.csv')
tariffs = pd.read_csv('/datasets/tariffs.csv')
users = pd.read_csv('/datasets/users.csv')

pd.options.display.max_columns = 500 # чтобы отображались все столбцы, и мы ничего не потеряли

Выгрузим общую информацию и первые пять строк каждого датафрейма, чтобы посмотреть все ли у нас хорошо с табуляцией, и какие данные у нас есть, чтобы планировать дальнейшие действия.
Заключение напишем в выводе по разделу.

In [3]:
calls.info()
calls.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202607 entries, 0 to 202606
Data columns (total 4 columns):
id           202607 non-null object
call_date    202607 non-null object
duration     202607 non-null float64
user_id      202607 non-null int64
dtypes: float64(1), int64(1), object(2)
memory usage: 6.2+ MB


,id,call_date,duration,user_id
0,1000_0,2018-07-25,0.00,1000
1,1000_1,2018-08-17,0.00,1000
2,1000_2,2018-06-11,2.85,1000
3,1000_3,2018-09-21,13.80,1000
4,1000_4,2018-12-15,5.18,1000
5,1000_5,2018-11-02,0.00,1000
6,1000_6,2018-10-18,0.00,1000
7,1000_7,2018-08-22,18.31,1000
8,1000_8,2018-09-15,18.44,1000
9,1000_9,2018-08-15,0.00,1000



<details>
<summary><b>Таблица `calls` (информация о звонках)</b></summary>
    
- id — уникальный номер звонка
- call_date — дата звонка
- duration — длительность звонка в минутах
- user_id — идентификатор пользователя, сделавшего звонок
    
</details>

In [4]:
internet.info()
internet.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149396 entries, 0 to 149395
Data columns (total 4 columns):
id              149396 non-null object
mb_used         149396 non-null float64
session_date    149396 non-null object
user_id         149396 non-null int64
dtypes: float64(1), int64(1), object(2)
memory usage: 4.6+ MB


,id,mb_used,session_date,user_id
0,1000_0,112.95,2018-11-25,1000
1,1000_1,1052.81,2018-09-07,1000
2,1000_2,1197.26,2018-06-25,1000
3,1000_3,550.27,2018-08-22,1000
4,1000_4,302.56,2018-09-24,1000
5,1000_5,399.97,2018-10-02,1000
6,1000_6,540.08,2018-09-07,1000
7,1000_7,415.70,2018-07-11,1000
8,1000_8,505.40,2018-10-08,1000
9,1000_9,345.54,2018-09-03,1000


<details>
<summary><b>Таблица `internet` (информация об интернет-сессиях)</b></summary>
    
- id — уникальный номер сессии
- mb_used — объём потраченного за сессию интернет-трафика (в мегабайтах)
- session_date — дата интернет-сессии
- user_id — идентификатор пользователя

</details>

In [5]:
messages.info()
messages.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123036 entries, 0 to 123035
Data columns (total 3 columns):
id              123036 non-null object
message_date    123036 non-null object
user_id         123036 non-null int64
dtypes: int64(1), object(2)
memory usage: 2.8+ MB


,id,message_date,user_id
0,1000_0,2018-06-27,1000
1,1000_1,2018-10-08,1000
2,1000_2,2018-08-04,1000
3,1000_3,2018-06-16,1000
4,1000_4,2018-12-05,1000
5,1000_5,2018-06-20,1000
6,1000_6,2018-11-19,1000
7,1000_7,2018-10-29,1000
8,1000_8,2018-06-25,1000
9,1000_9,2018-12-18,1000


<details>
<summary><b>Таблица `messages` (информация о сообщениях)</b></summary>
    
- id — уникальный номер сообщения
- message_date — дата сообщения
- user_id — идентификатор пользователя, отправившего сообщение
    
</details>

In [6]:
tariffs.info()
tariffs.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 8 columns):
messages_included        2 non-null int64
mg_per_month_included    2 non-null int64
minutes_included         2 non-null int64
rub_monthly_fee          2 non-null int64
rub_per_gb               2 non-null int64
rub_per_message          2 non-null int64
rub_per_minute           2 non-null int64
tariff_name              2 non-null object
dtypes: int64(7), object(1)
memory usage: 256.0+ bytes


,messages_included,mg_per_month_included,minutes_included,rub_monthly_fee,rub_per_gb,rub_per_message,rub_per_minute,tariff_name
0,50,15360,500,550,200,3,3,smart
1,1000,30720,3000,1950,150,1,1,ultra


Таблица `tariffs` содержит всего две строки и носит информационный характер, все танные представлены в читабельном виде.
В предобработке датафрейм не нуждается.

<details>
<summary><b>Таблица `tariffs` (информация о тарифах)</b></summary>
    
- tariff_name — название тарифа
- rub_monthly_fee — ежемесячная абонентская плата в рублях
- minutes_included — количество минут разговора в месяц, включённых в абонентскую плату
- messages_included — количество сообщений в месяц, включённых в абонентскую плату
- mb_per_month_included — объём интернет-трафика, включённого в абонентскую плату (в мегабайтах)
- rub_per_minute — стоимость минуты разговора сверх тарифного пакета (например, если в тарифе 100 минут разговора в месяц, то со 101 минуты будет взиматься плата)
- rub_per_message — стоимость отправки сообщения сверх тарифного пакета
- rub_per_gb — стоимость дополнительного гигабайта интернет-трафика сверх тарифного пакета (1 гигабайт = 1024 мегабайта)

</details>

<p>

<details>
<summary><b>Описание тарифов</b></summary>
<p>
<b>Тариф «Смарт»</b>

- Ежемесячная плата: 550 рублей
- Включено 500 минут разговора, 50 сообщений и 15 Гб интернет-трафика 

Стоимость услуг сверх тарифного пакета:
- минута разговора: 3 рубля
- сообщение: 3 рубля
- 1 Гб интернет-трафика: 200 рублей


<b>Тариф «Ультра»</b>

- Ежемесячная плата: 1950 рублей
- Включено 3000 минут разговора, 1000 сообщений и 30 Гб интернет-трафика 

Стоимость услуг сверх тарифного пакета:
- минута разговора: 1 рубль
- сообщение: 1 рубль
- 1 Гб интернет-трафика: 150 рублей
</p>
</details>

Обратите внимание: «Мегалайн» **всегда округляет вверх значения минут и мегабайтов.** Если пользователь проговорил всего 1 секунду, в тарифе засчитывается целая минута.

In [7]:
users.info()
users.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
user_id       500 non-null int64
age           500 non-null int64
churn_date    38 non-null object
city          500 non-null object
first_name    500 non-null object
last_name     500 non-null object
reg_date      500 non-null object
tariff        500 non-null object
dtypes: int64(2), object(6)
memory usage: 31.4+ KB


,user_id,age,churn_date,city,first_name,last_name,reg_date,tariff
0,1000,52,NaN,Краснодар,Рафаил,Верещагин,2018-05-25,ultra
1,1001,41,NaN,Москва,Иван,Ежов,2018-11-01,smart
2,1002,59,NaN,Стерлитамак,Евгений,Абрамович,2018-06-17,smart
3,1003,23,NaN,Москва,Белла,Белякова,2018-08-17,ultra
4,1004,68,NaN,Новокузнецк,Татьяна,Авдеенко,2018-05-14,ultra
5,1005,67,NaN,Набережные Челны,Афанасий,Горлов,2018-01-25,smart
6,1006,21,NaN,Ульяновск,Леонид,Ермолаев,2018-02-26,smart
7,1007,65,NaN,Москва,Юна,Березина,2018-04-19,smart
8,1008,63,NaN,Челябинск,Рустэм,Пономарёв,2018-12-19,smart
9,1009,24,NaN,Пермь,Василиса,Блинова,2018-03-22,smart


<details>
<summary><b>Таблица `users` (информация о пользователях)</b></summary>

- user_id — уникальный идентификатор пользователя
- first_name — имя пользователя
- last_name — фамилия пользователя
- age — возраст пользователя (годы)
- reg_date — дата подключения тарифа (день, месяц, год)
- churn_date — дата прекращения пользования тарифом (если значение пропущено, то тариф ещё действовал на момент выгрузки данных)
- city — город проживания пользователя
- tariff — название тарифного плана

</details>
<p>

**Вывод по разделу**

В первом шаге мы подключили все интересующие нас библиотеки, которые потребуются входе работы, прочитали файлы, проверили на содержание, и поставили себе первые задачи для изучения в части предобработки данных. "Грязную" работу будем выполнять постепенно в соответсвующих разделах.

Мы будем работать с пятью таблицами, в которых представлена информация о пользователях, звонках, сообщениях, интернет-сессиях и тарифах.

Если проанализировать все данные `info()` то можно обратить внимание, что явных пропусков (как минимум замеченных при беглом просмотре) не обнаружено. **Только в таблице `users` в столбце `churn_date`(дата прекращения пользования тарифом) имеются у нас пропуски, а это значит, что тариф ещё действовал на момент выгрузки данных.**

Таблица `tariffs` содержит всего две строки и носит информационный характер, все танные представлены в читабельном виде. В предобработке датафрейм не нуждается.

Обратите внимание: «Мегалайн» **всегда округляет вверх значения минут и мегабайтов.** Если пользователь проговорил всего 1 секунду, в тарифе засчитывается целая минута.

<div style="border:solid green 4px; padding: 20px">Отлично.</div>

---

### Шаг 2. Подготовьте данные

**Таблица `calls` (информация о звонках)**

Подготовим все таблицы постепенно к работе, начнем с `calls`.  
Проверим сначала на наличие дубликатов методом `.duplicated().sum()`.

In [8]:
calls.duplicated().sum() #ищем дубликаты

0

Дубликатов в таблице нет.

Столбец с датой совершения звонка у нас типа `object`, изменим его на `'datetime'` с помощью метода `to_datetime()`. А также добавим отдельный столбец с месяцем, нам потребуется дальше вывести данные по месяцам.

Столбец `duration` содержит длительность звонка в минутах, я бы не стала переводить данные в `int`, так как все таки секунды очень важны для точности.

Столбец `user_id`, мы видим пока только в одном значении, возможно упорядочены как раз по *id*.
Сформируем сводную таблицу и посмотрим какие у нас есть значения.

In [9]:
calls['call_date'] = pd.to_datetime(calls['call_date'], format='%Y-%m-%d') #меняем тип данных
calls['call_month'] = calls['call_date'].dt.month #месяц как январь = 1, декабрь = 12

In [10]:
duration_before = calls.pivot_table(index='duration', values='id', aggfunc='count')
duration_before

,id
duration,
0.00,39613
0.01,101
0.02,84
0.03,80
0.04,85
...,...
35.95,1
36.12,1
36.47,1


Уникальных значений у нас много вышло, интересно также, есть значения и до единицы (не 0.0). Оставим пока как есть.

Обратите внимание, что у большого количества звонков длительность — 0.0 минут. 
«Мегалайн» всегда округляет вверх значения минут и мегабайтов. А значит если пользователь проговорил всего 1 секунду, в тарифе засчитывается целая минута, а не округляется вниз. Интересно, что 

Также видим, что у нас есть значения **26.93**  или **30.58**. ВОзможно, 60 секунд приняты за единицу(100%), тогда чтобы восстановить справедливость в цифрах стоит количество секунд принять за 0.6 (60%).
Это проблема в данных, нужна предобработка. Более явных пропусков как можем заметить не имеем.

Поменяйем тип данных на `str`, чтобы произвести разделение столбца `duration` на два - `'min'` и `'sec'`, затем их переведем в `int` чтобы произвести математические вычисления (столбец `'sec'` помножим на **0.6**).
И далее сцепим два столбца обратно в `duration`.


In [11]:
calls['duration'] = calls['duration'].astype('str') # меняем тип на строковый
calls[['min','sec']] = calls['duration'].str.split('.',  n = 1, expand = True) # разделяем столбец на два новых
calls[['min','sec']] = calls[['min','sec']].astype('int') # заменяем тип данных на int для последующих вычислений
calls['sec'] = calls['sec'] * 0.6 # перемножаем секунды на 0.6
calls.head(10) # проверяем

,id,call_date,duration,user_id,call_month,min,sec
0,1000_0,2018-07-25,0.0,1000,7,0,0.0
1,1000_1,2018-08-17,0.0,1000,8,0,0.0
2,1000_2,2018-06-11,2.85,1000,6,2,51.0
3,1000_3,2018-09-21,13.8,1000,9,13,4.8
4,1000_4,2018-12-15,5.18,1000,12,5,10.8
5,1000_5,2018-11-02,0.0,1000,11,0,0.0
6,1000_6,2018-10-18,0.0,1000,10,0,0.0
7,1000_7,2018-08-22,18.31,1000,8,18,18.6
8,1000_8,2018-09-15,18.44,1000,9,18,26.4
9,1000_9,2018-08-15,0.0,1000,8,0,0.0


Наш столбец с секундами получился с типом данным `float`, мы же делили на 0.6. 

Переведем данные в `int` снова, значения должны будут округлиться вверх (в данном случае милисекунды до секунд) - исправим ситуацию с помощью метода `ceil()` библиотеки `numpy`.

In [12]:
calls['sec'] = np.ceil(calls['sec']).astype('int') # заменяем тип данных на int для последующих вычислений

Все встало на свои места, теперь вернем правильные значения в столбец `duration`, сцепим данные в ячейку, при ссылке на столбец допишем метод `.map(str)`, чтобы у нас случайно не сложились наши цифры.

In [13]:
calls['duration'] = calls['min'].map(str) + '.' + calls['sec'].map(str)
calls['duration'] = calls['duration'].astype('float') # переведем формат в float
del calls['min'] # удаляем наш рассчетный столбец
del calls['sec'] # удаляем наш рассчетный столбец
calls.head(10)

,id,call_date,duration,user_id,call_month
0,1000_0,2018-07-25,0.00,1000,7
1,1000_1,2018-08-17,0.00,1000,8
2,1000_2,2018-06-11,2.51,1000,6
3,1000_3,2018-09-21,13.50,1000,9
4,1000_4,2018-12-15,5.11,1000,12
5,1000_5,2018-11-02,0.00,1000,11
6,1000_6,2018-10-18,0.00,1000,10
7,1000_7,2018-08-22,18.19,1000,8
8,1000_8,2018-09-15,18.27,1000,9
9,1000_9,2018-08-15,0.00,1000,8


In [14]:
duration_after = calls.pivot_table(index='duration', values='id', aggfunc='count')
duration_after

,id
duration,
0.00,39613
0.10,268
0.11,183
0.12,93
0.13,67
...,...
35.80,1
36.29,1
36.80,1


Вернемся к тому, что у большого количества звонков длительность — 0.0 минут. Предположительно, это ошибка системы, ранее говорилось, что все звонки по несколько секунд округляются вверх. Чисто гипотетически, звонки были совершены, проверим насколько вероятно что звонок был средней или медианной длительности. 

Посмотрим отдельно значения среднего значения и медианы, посмотрим каким значением было бы правильней заменить. 

In [15]:
mean = calls['duration'].mean()
median = calls['duration'].median()

mean, median

(6.657817548258451, 6.0)

In [16]:
calls['duration'].describe()

count    202607.000000
mean          6.657818
std           5.817342
min           0.000000
25%           1.260000
50%           6.000000
75%          10.490000
max          38.000000
Name: duration, dtype: float64

Медиана меньше среднего значения, а это значит, что данные в выборке скошены вправо. Оно и логично, данные у нас все положительные. Более редкие значения расположены по возрастанию. 
Рассчитаем дисперсию, чтобы принять решение о замене.

In [17]:
variance_calls = np.var(calls['duration'], ddof=0)
variance_calls

33.84130180479198

Слишком большая дисперсия приводит к ошибкам, и если вдуматься, то заменить на усредненное или медианное значение будет неправильным. 
Также странно, что наша выборка содержит звонки и по несколько секунд. А вот эти звонки не прошли, вероятно они длились доли милисекунд и программа их не округлила до секунд.. Давайте заменим наши нули на минуту разговора.
Произведем замену посредством метода `replace`.

In [18]:
calls['duration'] = calls['duration'].replace(0.00, 1.00)

Обратимся к условию:

> **Обратите внимание:** «Мегалайн» всегда округляет вверх значения минут и мегабайтов. Если пользователь проговорил всего 1 секунду, в тарифе засчитывается целая минута.

Нам нужно все звонки округлить вверх до минуты, ну и перевести данные в `int`. Если мы переведем в `int` сразу, то данные округлятся вниз. Воспользуемся сначала методом `np.ceil()` а затем изменим тип на `'int'`.

In [19]:
calls['duration'] = np.ceil(calls['duration']).astype('int')
calls.info() #проверbм смену типа данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202607 entries, 0 to 202606
Data columns (total 5 columns):
id            202607 non-null object
call_date     202607 non-null datetime64[ns]
duration      202607 non-null int64
user_id       202607 non-null int64
call_month    202607 non-null int64
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 7.7+ MB


Основные манипуляции с данным датафреймом мы выполнили - тип данных изменен, новый столбец добавлен, и данные приведены в порядок; рассмотрим следующую таблицу `internet`.

**Таблица `internet` (информация об интернет-сессиях)**

In [20]:
internet.duplicated().sum() #ищем дубликаты

0

В столбце `mb_used` — объём потраченного за сессию интернет-трафика у нас формат `float`, оставим его, т.к. данные у нас в мегабайтах, важны будут все значения после запятой.
Столбец `session_date` также по аналогии со столбцом `calls['call_date']` переводим в формат `datetime`.
И добавляем столбец с месяцем сессии, эта информация потребуется в дальнейшем.

In [21]:
internet['session_date'] = pd.to_datetime(internet['session_date'], format='%Y-%m-%d') #меняем тип данных
internet['session_month'] = internet['session_date'].dt.month #месяц как январь = 1, декабрь = 12

Выгрузим количество уникальных значений столбца `mb_used`, чтобы исключить отсутствующие значения посредством `groupby`, по аналогии как мы делали данные сводной таблицей (`pivot_table()`).

In [22]:
internet.groupby('mb_used')['id'].count()

mb_used
0.00       19598
0.03           1
0.04           1
0.06           1
0.07           2
           ...  
1623.92        1
1651.25        1
1651.55        1
1702.75        1
1724.83        1
Name: id, Length: 70003, dtype: int64

Кажется, ситуация аналогична длительности звонков - у большого количества интернет-сессий длительность — 0.0 мб, «Мегалайн» округляет вверх значения минут и мегабайтов. 

Но мы видим, что система отлично разпознает и запоминает интернет-сессии по 0.03.
Снова напомним условие:

> **Обратите внимание:** «Мегалайн» всегда округляет вверх значения минут и мегабайтов. Если пользователь проговорил всего 1 секунду, в тарифе засчитывается целая минута.

<span style='color: purple;'><i><b>Абсолютно нулевые значения могут говорить, о том, что интернет был включен у пользователей, но им никто не пользовался. Интернет-сессия начата, но данные не передавались, как исходящие и входящие.</b></i></span> Оставим нули как есть, это тоже важно, нашим исследованиям они не помешают.

Так как стоимость при перерасходе у нас у нас в гигабайтах указана, то логичней посчитать использованный трафик сразу в гигабайтах.

Нам нужно сначала все интернет-сессии округлить вверх до целого, ну и перевести данные в `int`. Если мы переведем в `int` сразу, то данные округлятся вниз. Воспользуемся сначала методом `np.ceil()` а затем изменим тип на `'int'`.

In [23]:
internet['mb_used'] = np.ceil(internet['mb_used']).astype('int')

Проверим датафрейм.

In [24]:
internet.info() #проверbм смену типа данных
internet.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149396 entries, 0 to 149395
Data columns (total 5 columns):
id               149396 non-null object
mb_used          149396 non-null int64
session_date     149396 non-null datetime64[ns]
user_id          149396 non-null int64
session_month    149396 non-null int64
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 5.7+ MB


,id,mb_used,session_date,user_id,session_month
0,1000_0,113,2018-11-25,1000,11
1,1000_1,1053,2018-09-07,1000,9
2,1000_2,1198,2018-06-25,1000,6
3,1000_3,551,2018-08-22,1000,8
4,1000_4,303,2018-09-24,1000,9
5,1000_5,400,2018-10-02,1000,10
6,1000_6,541,2018-09-07,1000,9
7,1000_7,416,2018-07-11,1000,7
8,1000_8,506,2018-10-08,1000,10
9,1000_9,346,2018-09-03,1000,9


<div style="border:solid green 4px; padding: 20px">Пока всё круто.</div>

**Таблица `messages` (информация о сообщениях)**

Посмотрим датафрейм `messages`, выгружаем снова общую инфо и первые 10 строчек, анализируем.

In [25]:
messages.info()
messages.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123036 entries, 0 to 123035
Data columns (total 3 columns):
id              123036 non-null object
message_date    123036 non-null object
user_id         123036 non-null int64
dtypes: int64(1), object(2)
memory usage: 2.8+ MB


,id,message_date,user_id
0,1000_0,2018-06-27,1000
1,1000_1,2018-10-08,1000
2,1000_2,2018-08-04,1000
3,1000_3,2018-06-16,1000
4,1000_4,2018-12-05,1000
5,1000_5,2018-06-20,1000
6,1000_6,2018-11-19,1000
7,1000_7,2018-10-29,1000
8,1000_8,2018-06-25,1000
9,1000_9,2018-12-18,1000


In [26]:
messages.duplicated().sum() #ищем дубликаты

0

Особых проблем не видим, меняем тип данных в столбце, методом `to_datetime`  и добавляем новый столбец тут же с месяцем.

In [27]:
messages['message_date'] = pd.to_datetime(messages['message_date'], format='%Y-%m-%d') #меняем тип данных
messages['message_month'] = messages['message_date'].dt.month #месяц как январь = 1, декабрь = 12

**Таблица `tariffs` (информация о тарифах)**

Для удобства переименуем столбец `mg_per_month_included` в `mg_included`.

In [28]:
tariffs.columns = ['messages_included', 'mg_included', 'minutes_included',
       'rub_monthly_fee', 'rub_per_gb', 'rub_per_message', 'rub_per_minute',
       'tariff_name']

Следом открываем последний датафрем с информацией по пользователям.

**Таблица `users` (информация о пользователях)**

In [29]:
users.info()
users.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
user_id       500 non-null int64
age           500 non-null int64
churn_date    38 non-null object
city          500 non-null object
first_name    500 non-null object
last_name     500 non-null object
reg_date      500 non-null object
tariff        500 non-null object
dtypes: int64(2), object(6)
memory usage: 31.4+ KB


,user_id,age,churn_date,city,first_name,last_name,reg_date,tariff
0,1000,52,NaN,Краснодар,Рафаил,Верещагин,2018-05-25,ultra
1,1001,41,NaN,Москва,Иван,Ежов,2018-11-01,smart
2,1002,59,NaN,Стерлитамак,Евгений,Абрамович,2018-06-17,smart
3,1003,23,NaN,Москва,Белла,Белякова,2018-08-17,ultra
4,1004,68,NaN,Новокузнецк,Татьяна,Авдеенко,2018-05-14,ultra
5,1005,67,NaN,Набережные Челны,Афанасий,Горлов,2018-01-25,smart
6,1006,21,NaN,Ульяновск,Леонид,Ермолаев,2018-02-26,smart
7,1007,65,NaN,Москва,Юна,Березина,2018-04-19,smart
8,1008,63,NaN,Челябинск,Рустэм,Пономарёв,2018-12-19,smart
9,1009,24,NaN,Пермь,Василиса,Блинова,2018-03-22,smart


In [30]:
users.duplicated().sum() #ищем дубликаты

0

У нас всего 38 заполненных значений в столбце `churn_date`, здесь отражается дата прекращения пользования тарифом, а значит, что если значение пропущено, то тариф ещё действовал на момент выгрузки данных.
Так как обработкой мы занимаемся сейчас, заменим данные на текущую дату.

А также поменяем формат столбца сразу и выведем новый столбец с месяцем.

In [31]:
users['churn_date'] = users['churn_date'].fillna(value='2019-11-26')
users['churn_date'] = pd.to_datetime(users['churn_date'], format='%Y-%m-%d') 
users['churn_month'] = users['churn_date'].dt.month #месяц как январь = 1, декабрь = 12

У нас в таблицах `users` и `tariffs` разнятся наименования столбцов, переименуем на `tariff_name`.
И с помощью метода `merge()` добавим таблицу `tariffs` по столбцу `tariff_name`.

In [32]:
users.columns = ['user_id', 'age', 'churn_date', 'city', 'first_name', 'last_name',
       'reg_date', 'tariff_name', 'churn_month']

users = users.merge(tariffs, on='tariff_name', how='left')

In [33]:
users.info() #проверяем замену

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 0 to 499
Data columns (total 16 columns):
user_id              500 non-null int64
age                  500 non-null int64
churn_date           500 non-null datetime64[ns]
city                 500 non-null object
first_name           500 non-null object
last_name            500 non-null object
reg_date             500 non-null object
tariff_name          500 non-null object
churn_month          500 non-null int64
messages_included    500 non-null int64
mg_included          500 non-null int64
minutes_included     500 non-null int64
rub_monthly_fee      500 non-null int64
rub_per_gb           500 non-null int64
rub_per_message      500 non-null int64
rub_per_minute       500 non-null int64
dtypes: datetime64[ns](1), int64(10), object(5)
memory usage: 66.4+ KB


---

**Посчитаем для каждого пользователя:**
- количество сделанных звонков и израсходованных минут разговора по месяцам;
- количество отправленных сообщений по месяцам;
- объем израсходованного интернет-трафика по месяцам;
- помесячную выручку с каждого пользователя (вычтите бесплатный лимит из суммарного количества звонков, сообщений и интернет-трафика; остаток умножьте на значение из тарифного плана; прибавьте абонентскую плату, соответствующую тарифному плану).

Делаем три сводных таблицы по звонкам, сообщениям и трафику в которым отразим инормацию помесячно.

При применении параментра `margins` у нас добавляется не только столбец, но и строчка с итогом, поэтому строку уберем методом `iloc` и тут же пересохраним таблицы.

In [34]:
calls_monthly = calls.pivot_table(index='user_id', # для каждого пользователя
                                columns='call_month', # по месяцам
                                values='duration', # израсходованных минут
                                aggfunc='sum', 
                                margins=True, # суммируем столбцы
                                margins_name='total').iloc[:-1] # присваиваем имя столбцу

messages_monthly = messages.pivot_table(index='user_id', # для каждого пользователя
                                      columns='message_month', # по месяцам
                                      values='id', # подсчет сообщений
                                      aggfunc='count', 
                                      margins=True, 
                                      margins_name='total').iloc[:-1] # присваиваем имя столбцу

internet_monthly = internet.pivot_table(index='user_id', # для каждого пользователя
                                      columns='session_month', # по месяцам
                                      values='mb_used', # израсходованного трафика
                                      aggfunc='sum', 
                                      margins=True, # суммируем столбцы
                                      margins_name='total').iloc[:-1] # присваиваем имя столбцу

# убираем строку total с помощью .iloc[:-1], останется только столбец
#calls_monthly = calls_monthly.iloc[:-1]
#messages_monthly = messages_monthly.iloc[:-1]
#internet_monthly = internet_monthly.iloc[:-1]

Добавим наименования тарифов к таблицам, а затем остальные столбцы для последующей организации подсчетов выручки.
Тут же сделаем копии для дальнейшей работы, к таблицам `monthly` вернемся позже.
Оформим для удобства через цикл.

In [35]:
tariff = users[['user_id', 'tariff_name']] 

calls_monthly = calls_monthly.merge(tariff, on='user_id', how='left')
messages_monthly = messages_monthly.merge(tariff, on='user_id', how='left')
internet_monthly = internet_monthly.merge(tariff, on='user_id', how='left')

calls_stats = calls_monthly.copy()
messages_stats = messages_monthly.copy()
internet_stats = internet_monthly.copy()

Сделаем таблицы на основе `tariffs` и только после методом `merge()` свяжем таблицы для дальнейших частичных расчетов выручки.

In [36]:
calls_tariffs = tariffs[['tariff_name', 'minutes_included', 'rub_per_minute']]
messages_tariffs = tariffs[['tariff_name', 'messages_included', 'rub_per_message']]
internet_tariffs = tariffs[['tariff_name', 'mg_included', 'rub_per_gb']]
# взяли только необходимые столбцы, которые потребуются для вычислений 
# и добавим их нашей сводной таблице
calls_stats = calls_stats.merge(calls_tariffs, on='tariff_name', how='left')
messages_stats = messages_stats.merge(messages_tariffs, on='tariff_name', how='left')
internet_stats = internet_stats.merge(internet_tariffs, on='tariff_name', how='left')

Облегчим дальнейшую функцию и переименуем столбцы наших датафреймов.

In [37]:
calls_stats.columns = ['user_id', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                       'total', 'tariff_name', 'limit', 'price']
messages_stats.columns = ['user_id', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                       'total', 'tariff_name', 'limit', 'price']
internet_stats.columns = ['user_id', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                       'total', 'tariff_name', 'limit', 'price']

In [38]:
calls_stats.columns # проверяем наименования столбцов

Index([    'user_id',             1,             2,             3,
                   4,             5,             6,             7,
                   8,             9,            10,            11,
                  12,       'total', 'tariff_name',       'limit',
             'price'],
      dtype='object')

In [39]:
internet_stats.columns # проверяем наименования столбцов

Index([    'user_id',             1,             2,             3,
                   4,             5,             6,             7,
                   8,             9,            10,            11,
                  12,       'total', 'tariff_name',       'limit',
             'price'],
      dtype='object')

Посчитаем посмесячно сколько пользователи тратят за перерасход лимитов тарифа.  Вычтем бесплатный лимит из суммарного расхода; остаток умножим на значение из тарифного плана; Прибавлять абонентскую плату, соответствующую тарифному плану будем потом.

Для этого напишем функцию, которая будет создавать новый столбец с для каждого месяца и просчитывать выручку. 

У нас есть наверняка клиенты, которые укладываются в лимиты, соответственно выручки не будет. Заменим тут же отрицательные значения на нули, чтобы не было неловких ситуаций.

In [40]:
col_month = [x for x in range(1, 13)] # выбираем столбцы для функции, январь-декабрь
for i in col_month:
    internet_stats[i] = internet_stats[i] / 1024
    
internet_stats['limit'] = internet_stats['limit'] / 1024

In [41]:
def revenue(data):
    y = data['limit']
    z = data['price']
    
    for col in col_month:
        new_name = 'overpayment_' + str(col) # создаем новое имя столбцамб overpayment c англ. переплата
        data[new_name] = ((data[col] - y) * z) 
        data.loc[data[new_name] < 0, new_name] = 0 # все отрицательные значения заменяем на 0
# из суммарной продолжительности звонка вычитаем бесплатный лимит. остаток умножаем на значения из тарифного плана
# после прибавляем абонентскую плату по тарифу

revenue(calls_stats)
revenue(messages_stats)
revenue(internet_stats)

Проверим работу функции, если все успешно, то добавим общий столбец с переплатой за год.
Проверять будем на примере одной из таблиц.

In [42]:
internet_stats.head(10)

,user_id,1,2,3,4,5,6,7,8,9,10,11,12,total,tariff_name,limit,price,overpayment_1,overpayment_2,overpayment_3,overpayment_4,overpayment_5,overpayment_6,overpayment_7,overpayment_8,overpayment_9,overpayment_10,overpayment_11,overpayment_12
0,1000,NaN,NaN,NaN,NaN,2.203125,22.711914,13.687500,13.740234,14.239258,14.372070,14.421875,9.596680,107492,ultra,30.0,150,NaN,NaN,NaN,NaN,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.019531,13.723633,32505,smart,15.0,200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,603.906250,0.000000
2,1002,NaN,NaN,NaN,NaN,NaN,10.616211,17.188477,19.870117,16.320312,13.579102,18.173828,17.711914,116183,smart,15.0,200,NaN,NaN,NaN,NaN,NaN,0.00000,437.695312,974.023438,264.062500,0.000000,634.765625,542.382812
3,1003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.370117,12.186523,14.432617,11.101562,9.894531,57329,ultra,30.0,150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000
4,1004,NaN,NaN,NaN,NaN,13.100586,17.206055,21.721680,27.936523,14.770508,18.060547,15.264648,17.615234,149172,ultra,30.0,150,NaN,NaN,NaN,NaN,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,1005,2.666016,7.220703,10.856445,9.145508,7.692383,9.643555,7.522461,7.376953,6.862305,7.625977,12.384766,8.476562,99813,smart,15.0,200,0.0,0.0,0.000000,0.0,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,1006,NaN,1.362305,16.903320,13.626953,12.679688,12.097656,19.531250,18.874023,15.178711,19.259766,12.963867,18.064453,164395,smart,15.0,200,NaN,0.0,380.664062,0.0,0.00000,0.00000,906.250000,774.804688,35.742188,851.953125,0.000000,612.890625
7,1007,NaN,NaN,NaN,7.842773,17.292969,13.353516,15.312500,20.237305,12.333984,14.105469,16.005859,12.218750,131792,smart,15.0,200,NaN,NaN,NaN,0.0,458.59375,0.00000,62.500000,1047.460938,0.000000,0.000000,201.171875,0.000000
8,1008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.925781,13236,smart,15.0,200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
9,1009,NaN,NaN,7.186523,14.384766,13.072266,15.707031,24.171875,23.181641,21.439453,19.640625,16.471680,18.205078,177624,smart,15.0,200,NaN,NaN,0.000000,0.0,0.00000,141.40625,1834.375000,1636.328125,1287.890625,928.125000,294.335938,641.015625


<div style="border:solid green 4px; padding: 20px">Хорошо.</div>

In [43]:
col_overpay = []
for i in range(1,13):
    col_overpay.append('overpayment_{}'.format(i))
calls_stats['overpayment_calls'] = calls_stats[col_overpay].sum(axis=1)
messages_stats['overpayment_mess'] = messages_stats[col_overpay].sum(axis=1)
internet_stats['overpayment_int'] = internet_stats[col_overpay].sum(axis=1)

Самое время добавить данные по перерасходам добавить в таблицу  с данными по пользователям `users`.
Перенесем столбцы в рабочую таблицу.

In [44]:
transfer_calls = calls_stats[['user_id',
                              'overpayment_1',
                              'overpayment_2',
                              'overpayment_3',
                              'overpayment_4',
                              'overpayment_5',
                              'overpayment_6',
                              'overpayment_7',
                              'overpayment_8',
                              'overpayment_9',
                              'overpayment_10',
                              'overpayment_11',
                              'overpayment_12', 
                              'overpayment_calls']]
transfer_mess = messages_stats[['user_id', 
                                'overpayment_1',
                              'overpayment_2',
                              'overpayment_3',
                              'overpayment_4',
                              'overpayment_5',
                              'overpayment_6',
                              'overpayment_7',
                              'overpayment_8',
                              'overpayment_9',
                              'overpayment_10',
                              'overpayment_11',
                              'overpayment_12',
                                'overpayment_mess']]
transfer_int = internet_stats[['user_id', 
                               'overpayment_1',
                              'overpayment_2',
                              'overpayment_3',
                              'overpayment_4',
                              'overpayment_5',
                              'overpayment_6',
                              'overpayment_7',
                              'overpayment_8',
                              'overpayment_9',
                              'overpayment_10',
                              'overpayment_11',
                              'overpayment_12',
                               'overpayment_int']]

telecom_stats = (
    users
    .merge(transfer_calls, on='user_id', how='left')
    .merge(transfer_mess, on='user_id', how='left')
    .merge(transfer_int, on='user_id', how='left')
)


Посмотрим как встали столбцы в `telecom_stats` - у нас были таблицы с одинаковыми именами.

In [45]:
telecom_stats.columns

Index(['user_id', 'age', 'churn_date', 'city', 'first_name', 'last_name',
       'reg_date', 'tariff_name', 'churn_month', 'messages_included',
       'mg_included', 'minutes_included', 'rub_monthly_fee', 'rub_per_gb',
       'rub_per_message', 'rub_per_minute', 'overpayment_1_x',
       'overpayment_2_x', 'overpayment_3_x', 'overpayment_4_x',
       'overpayment_5_x', 'overpayment_6_x', 'overpayment_7_x',
       'overpayment_8_x', 'overpayment_9_x', 'overpayment_10_x',
       'overpayment_11_x', 'overpayment_12_x', 'overpayment_calls',
       'overpayment_1_y', 'overpayment_2_y', 'overpayment_3_y',
       'overpayment_4_y', 'overpayment_5_y', 'overpayment_6_y',
       'overpayment_7_y', 'overpayment_8_y', 'overpayment_9_y',
       'overpayment_10_y', 'overpayment_11_y', 'overpayment_12_y',
       'overpayment_mess', 'overpayment_1', 'overpayment_2', 'overpayment_3',
       'overpayment_4', 'overpayment_5', 'overpayment_6', 'overpayment_7',
       'overpayment_8', 'overpayment_9', 'ove

In [46]:
rub_monthly_fee = telecom_stats['rub_monthly_fee']

for i in range(1,13):
    telecom_stats['revenue_{:.0f}'.format(i)] = telecom_stats['overpayment_{}_x'.format(i)] \
    + telecom_stats['overpayment_{}_y'.format(i)] + telecom_stats['overpayment_{}'.format(i)] + rub_monthly_fee

In [47]:
telecom_stats

,user_id,age,churn_date,city,first_name,last_name,reg_date,tariff_name,churn_month,messages_included,mg_included,minutes_included,rub_monthly_fee,rub_per_gb,rub_per_message,rub_per_minute,overpayment_1_x,overpayment_2_x,overpayment_3_x,overpayment_4_x,overpayment_5_x,overpayment_6_x,overpayment_7_x,overpayment_8_x,overpayment_9_x,overpayment_10_x,overpayment_11_x,overpayment_12_x,overpayment_calls,overpayment_1_y,overpayment_2_y,overpayment_3_y,overpayment_4_y,overpayment_5_y,overpayment_6_y,overpayment_7_y,overpayment_8_y,overpayment_9_y,overpayment_10_y,overpayment_11_y,overpayment_12_y,overpayment_mess,overpayment_1,overpayment_2,overpayment_3,overpayment_4,overpayment_5,overpayment_6,overpayment_7,overpayment_8,overpayment_9,overpayment_10,overpayment_11,overpayment_12,overpayment_int,revenue_1,revenue_2,revenue_3,revenue_4,revenue_5,revenue_6,revenue_7,revenue_8,revenue_9,revenue_10,revenue_11,revenue_12
0,1000,52,2019-11-26,Краснодар,Рафаил,Верещагин,2018-05-25,ultra,11,1000,30720,3000,1950,150,1,1,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,1950.00000,1950.000000,1950.000000,1950.000000,1950.000000,1950.000000,1950.000000,1950.000000
1,1001,41,2019-11-26,Москва,Иван,Ежов,2018-11-01,smart,11,50,15360,500,550,200,3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,603.906250,0.000000,603.906250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1002,59,2019-11-26,Стерлитамак,Евгений,Абрамович,2018-06-17,smart,11,50,15360,500,550,200,3,3,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.000000,437.695312,974.023438,264.062500,0.000000,634.765625,542.382812,2852.929688,NaN,NaN,NaN,NaN,NaN,550.000000,987.695312,1524.023438,814.062500,550.000000,1184.765625,1092.382812
3,1003,23,2019-11-26,Москва,Белла,Белякова,2018-08-17,ultra,11,1000,30720,3000,1950,150,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1950.000000,1950.000000,1950.000000,1950.000000,1950.000000
4,1004,68,2019-11-26,Новокузнецк,Татьяна,Авдеенко,2018-05-14,ultra,11,1000,30720,3000,1950,150,1,1,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,1950.00000,1950.000000,1950.000000,1950.000000,1950.000000,1950.000000,1950.000000,1950.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1495,65,2019-11-26,Иркутск,Авксентий,Фокин,2018-08-28,ultra,11,1000,30720,3000,1950,150,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,349.365234,0.000000,1208.789062,0.000000,1558.154297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1950.000000,2299.365234,1950.000000,3158.789062,1950.000000
496,1496,36,2019-11-26,Вологда,Трифон,Блохин,2018-01-27,smart,11,50,15360,500,550,200,3,3,0.0,318.0,207.0,108.0,0.0,216.0,276.0,159.0,0.0,0.0,528.0,99.0,1911.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,2570.898438,1880.859375,1120.703125,1636.71875,938.085938,1764.648438,1601.562500,2267.578125,2510.742188,1900.781250,1054.101562,19246.679688,550.0,3438.898438,2637.859375,1778.703125,2186.71875,1704.085938,2590.648438,2310.562500,2817.578125,3060.742188,2978.781250,1706.101562
497,1497,32,2019-11-26,Челябинск,Кар

In [48]:
col_revenue = []
for i in range(1,13):
    col_revenue.append('revenue_{}'.format(i))
telecom_stats['revenue'] = telecom_stats[col_revenue].sum(axis=1)
telecom_stats

,user_id,age,churn_date,city,first_name,last_name,reg_date,tariff_name,churn_month,messages_included,mg_included,minutes_included,rub_monthly_fee,rub_per_gb,rub_per_message,rub_per_minute,overpayment_1_x,overpayment_2_x,overpayment_3_x,overpayment_4_x,overpayment_5_x,overpayment_6_x,overpayment_7_x,overpayment_8_x,overpayment_9_x,overpayment_10_x,overpayment_11_x,overpayment_12_x,overpayment_calls,overpayment_1_y,overpayment_2_y,overpayment_3_y,overpayment_4_y,overpayment_5_y,overpayment_6_y,overpayment_7_y,overpayment_8_y,overpayment_9_y,overpayment_10_y,overpayment_11_y,overpayment_12_y,overpayment_mess,overpayment_1,overpayment_2,overpayment_3,overpayment_4,overpayment_5,overpayment_6,overpayment_7,overpayment_8,overpayment_9,overpayment_10,overpayment_11,overpayment_12,overpayment_int,revenue_1,revenue_2,revenue_3,revenue_4,revenue_5,revenue_6,revenue_7,revenue_8,revenue_9,revenue_10,revenue_11,revenue_12,revenue
0,1000,52,2019-11-26,Краснодар,Рафаил,Верещагин,2018-05-25,ultra,11,1000,30720,3000,1950,150,1,1,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,1950.00000,1950.000000,1950.000000,1950.000000,1950.000000,1950.000000,1950.000000,1950.000000,15600.000000
1,1001,41,2019-11-26,Москва,Иван,Ежов,2018-11-01,smart,11,50,15360,500,550,200,3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,603.906250,0.000000,603.906250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
2,1002,59,2019-11-26,Стерлитамак,Евгений,Абрамович,2018-06-17,smart,11,50,15360,500,550,200,3,3,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.000000,437.695312,974.023438,264.062500,0.000000,634.765625,542.382812,2852.929688,NaN,NaN,NaN,NaN,NaN,550.000000,987.695312,1524.023438,814.062500,550.000000,1184.765625,1092.382812,6702.929688
3,1003,23,2019-11-26,Москва,Белла,Белякова,2018-08-17,ultra,11,1000,30720,3000,1950,150,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1950.000000,1950.000000,1950.000000,1950.000000,1950.000000,9750.000000
4,1004,68,2019-11-26,Новокузнецк,Татьяна,Авдеенко,2018-05-14,ultra,11,1000,30720,3000,1950,150,1,1,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,1950.00000,1950.000000,1950.000000,1950.000000,1950.000000,1950.000000,1950.000000,1950.000000,15600.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1495,65,2019-11-26,Иркутск,Авксентий,Фокин,2018-08-28,ultra,11,1000,30720,3000,1950,150,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,349.365234,0.000000,1208.789062,0.000000,1558.154297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1950.000000,2299.365234,1950.000000,3158.789062,1950.000000,11308.154297
496,1496,36,2019-11-26,Вологда,Трифон,Блохин,2018-01-27,smart,11,50,15360,500,550,200,3,3,0.0,318.0,207.0,108.0,0.0,216.0,276.0,159.0,0.0,0.0,528.0,99.0,1911.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,2570.898438,1880.859375,1120.703125,1636.71875,938.085938,1764.648438,1601.562500,2267.578125,2510.742188,1900.781250,1054.101562,19246.679688,550.0,3438.898438,2637.859375,1778.703125,2186.71875,1704.085938,2590.648438,2310.562500,

####  Вывод по разделу

Первым делом мы привели данные к нужным типам, нашли и исправили ошибки в данных.

Обратите внимание, что у большого количества звонков была длительность — 0.0 минут, мы заменили на 1 минуту.
Таже была проблема с представлением минут в длительности звонка, мы исправили формат представления (секунды) и уже подготовили данные для дальнейшей работы.

Согласно заданию мы посчитали для каждого пользователя:
- количество израсходованных минут разговора по месяцам;
- количество отправленных сообщений по месяцам;
- объем израсходованного интернет-трафика по месяцам;

Помесячную выручку с каждого пользователя включили в соответствующую статистику по звонкам, cообщениям и интернет-трафику. Выручка считалась - как разница суммарного количества звонков/сообщений/интернет-трафика и бесплатного лимита; остаток умножили на значение из тарифного плана; прибавили абонентскую плату, соответствующую тарифному плану. 

Данные сохранили в отдельной таблице `telecom_stats`.

---

### Шаг 3. Проанализируйте данные

Опишите поведение клиентов оператора, исходя из выборки. 
- Сколько минут разговора, сколько сообщений и какой объём интернет-трафика требуется пользователям каждого тарифа в месяц?
- Посчитайте среднее количество, дисперсию и стандартное отклонение. 
- Постройте гистограммы. 
- Опишите распределения.

Описательная статистика используется для описания основных характеристик данных в исследованиях. Они предоставляют простые сводки о данных и вместе с простым графическим анализом составляют основу каждого статистического анализа.

Следовательно, всегда полезно начинать с этих простых вычислений, прежде чем углубляться в построение сложных моделей.

Прежде чем будем выводить данные по запросу, настроим формат float чтобы мы ничего не потеряли и не выполняли рутину в вычислениях.

In [49]:
pd.options.display.float_format = '{:.2f}'.format

Выполним вычисления суммы израсходанного пользователями, средних значений, дисперсии и стандартного отклонения помесячно в таблицах `calls_stats`, `messages_stats`, `internet_stats`.

Напишем функцию для общих вычислений суммы, среднего, дисперсии и стандартного отклонения по каждому столбцу датафрейма.

In [50]:
def meaning(data):
    data.loc['total_all'] = data.sum()
    data.loc['mean_all'] = data.iloc[:-1, :].mean()

meaning(calls_monthly)
meaning(messages_monthly)
meaning(internet_monthly)    

def meaning_smart(data):
    namequery = str(data) + '_query'
    namequery =(
        data
        .query('tariff_name in "smart"')
    )
    data.loc['total_smart'] = namequery.sum()
    data.loc['mean_smart'] = namequery.mean()

meaning_smart(calls_monthly)
meaning_smart(messages_monthly)
meaning_smart(internet_monthly)

def meaning_ultra(data):
    namequery = str(data) + '_query'
    namequery =(
        data
        .query('tariff_name in "ultra"')
    )
    data.loc['total_ultra'] = namequery.sum()
    data.loc['mean_ultra'] = namequery.mean()

meaning_ultra(calls_monthly)
meaning_ultra(messages_monthly)
meaning_ultra(internet_monthly)

Мы добавили соответствующие столбцы, и теперь подготовим таблицы для построения гистограмм.
Для построения барчартов будем использовать функцию.

In [51]:
calls_hist = calls_monthly.tail(6)
calls_hist = calls_hist.iloc[:, :-2]
del calls_hist['user_id']
calls_hist = calls_hist.transpose()
calls_hist

,total_all,mean_all,total_smart,mean_smart,total_ultra,mean_ultra
1,11096.00,258.05,7128.00,209.65,3968.00,440.89
2,25078.00,305.83,17766.00,306.31,7312.00,304.67
3,49455.00,433.82,30841.00,400.53,18614.00,503.08
4,68078.00,412.59,42224.00,377.00,25854.00,487.81
5,91341.00,445.57,56175.00,398.40,35166.00,549.47
6,108063.00,442.88,66699.00,409.20,41364.00,510.67
7,135264.00,468.04,84277.00,419.29,50987.00,579.40
8,157929.00,468.63,101800.00,435.04,56129.00,544.94
9,178090.00,481.32,114421.00,441.78,63669.00,573.59
10,199317.00,484.96,128859.00,439.79,70458.00,597.10


In [55]:
month = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'avg', 'sep', 'oct', 'nov', 'dec'] # calls_hist.index

def figshow(smar, ultr, tota, title):
    fig = go.Figure(data=[
        go.Bar(name='smart', x=month, y=smar),
        go.Bar(name='ultra', x=month, y=ultr),
        go.Scatter(name='total', x=month, y=tota)       
    ])
    # вносим надписи данных
    fig.update_layout(barmode='group',
                      title_text=title,
                      xaxis={'title':'month'}, 
                      yaxis={'title':'mean'})
    fig.show()

In [56]:
figshow(calls_hist['mean_smart'], 
        calls_hist['mean_ultra'], 
        calls_hist['mean_all'], 
        'Распределение по звонкам (среднее)')

<div style="border:solid green 4px; padding: 20px">За верный и аккуратный график - спасибо!</div>

In [57]:
calls_hist['total_all'][12] //  calls_hist['total_all'][1]

22.0

В течении года длительность звонков в месяц только увеличивается, связано с тем, что количество клиентов увеличивается с течением года. Суммарная длительность звонков увеличилась в 22 раза.
В начале года, среднестатический пользователь тарифа Смарт проговаривал 197 минут, когда пользователи Утры проговаривали 400 минут в месяц. Пользователи в среднем не превышают ежемесячных лимитов. 

В феврале пользователи в принципе разговаривали меньше, но в остальном средняя продолжительность разговоров в месяц стабильна и держится на одном уровне. 

Клиенты тарифа Ультра как мы можем видеть не проговаривают 3 тысячи минут в месяц, а владельцы тарифа Смарт в среднем придерживаются лимитов и достаточно успешно.


In [58]:
messages_hist = messages_monthly.tail(6)
messages_hist = messages_hist.iloc[:, :-2]
del messages_hist['user_id']
messages_hist = messages_hist.transpose()
messages_hist

,total_all,mean_all,total_smart,mean_smart,total_ultra,mean_ultra
1,924.00,24.32,620.00,20.00,304.00,43.43
2,1924.00,29.15,1397.00,27.94,527.00,32.94
3,3648.00,38.81,2453.00,35.55,1195.00,47.80
4,5179.00,38.36,3443.00,35.86,1736.00,44.51
5,7298.00,41.94,4762.00,39.03,2536.00,48.77
6,8983.00,42.98,5478.00,38.58,3505.00,52.31
7,11320.00,45.83,6784.00,38.99,4536.00,62.14
8,13088.00,44.82,7905.00,38.75,5183.00,58.90
9,14759.00,46.41,8746.00,39.22,6013.00,63.29
10,17114.00,47.94,9927.00,39.08,7187.00,69.78


Количество сообщений также растет с высокой скоростью в течении года, давайте посмотрим сколько в среднем пользователей отправляли вообще сообщения.

In [59]:
messages_hist['users_mes'] = messages_hist['total_all'] / messages_hist['mean_all']

users_mes = go.Figure(go.Scatter(name='total', x=month, y=messages_hist['users_mes']))
users_mes.show()

И действительно в начале года сообщения отправили почти 40 человек, а к концу года почти 80% пользователей (~400 человек) из выборки активно чатились. Количество пользователей общающихся посредством смс увеличилось с начала года на 10%

In [60]:
figshow(messages_hist['mean_smart'], 
        messages_hist['mean_ultra'], 
        messages_hist['mean_all'], 
        'Распределение по сообщениям (среднее)')

Количество отправленных сообщений равномерно увеличивается с течением года. 
Пользовали с тарифом смарт пользуются с конца первого квартала стабильно на одном уровне сообщениями, а вот пользовали тарифа ультра каждый месяц используют возможности тарифного плана не всегдана одном уровне. Пик приходится на июль, конец года. Самое время отпусков и отчетного периода.

In [61]:
internet_hist = internet_monthly.tail(6)
internet_hist = internet_hist.iloc[:, :-2]
del internet_hist['user_id']
internet_hist = internet_hist.transpose()
internet_hist

,total_all,mean_all,total_smart,mean_smart,total_ultra,mean_ultra
1,408213.00,9493.33,289831.00,8524.44,118382.00,13153.56
2,956348.00,11954.35,673461.00,11611.40,282887.00,12858.50
3,1813960.00,15911.93,1164537.00,15123.86,649423.00,17551.97
4,2436111.00,14587.49,1509702.00,13479.48,926409.00,16843.80
5,3529893.00,17052.62,2231353.00,15825.20,1298540.00,19674.85
6,4184770.00,16942.39,2593622.00,15814.77,1591148.00,19170.46
7,5035299.00,17303.43,3168466.00,15763.51,1866833.00,20742.59
8,6005126.00,17662.14,3908561.00,16703.25,2096565.00,19778.92
9,6454931.00,17259.17,4244377.00,16324.53,2210554.00,19390.82
10,7400754.00,17790.27,4932898.00,16778.56,2467856.00,20228.33


In [62]:
figshow(internet_hist['mean_smart'], 
        internet_hist['mean_ultra'], 
        internet_hist['mean_all'], 
        'Распределение по трафику (среднее)')

А вот с использованием интернета все обстоит не так как, например, с теми же звонками и сообщениями.
Интернетом пользуются активно все пользователи, из месяца в месяц вне зависимости от количества пользователей средние значения находятся на одном уровне.
Если пользователи тарифа Ультра не превышают лимитов на интернет, то пользователи Смарт наоборот стабильно не превышают лимит на 1-3 гб.

---

Для нас важно оценить доходность за год, поэтому посчитаем в нашу таблицу среднее значение за год, дисперсию и стандартное отклонение по каждому из тарифов.
Напомним себе какие наименования столбцов у нас есть в таблице `telecom_stats`, а затем сформируем датафрейм для подсчета средних, дисперсии и стандратного отклонения.

In [63]:
telecom_stats.columns

Index(['user_id', 'age', 'churn_date', 'city', 'first_name', 'last_name',
       'reg_date', 'tariff_name', 'churn_month', 'messages_included',
       'mg_included', 'minutes_included', 'rub_monthly_fee', 'rub_per_gb',
       'rub_per_message', 'rub_per_minute', 'overpayment_1_x',
       'overpayment_2_x', 'overpayment_3_x', 'overpayment_4_x',
       'overpayment_5_x', 'overpayment_6_x', 'overpayment_7_x',
       'overpayment_8_x', 'overpayment_9_x', 'overpayment_10_x',
       'overpayment_11_x', 'overpayment_12_x', 'overpayment_calls',
       'overpayment_1_y', 'overpayment_2_y', 'overpayment_3_y',
       'overpayment_4_y', 'overpayment_5_y', 'overpayment_6_y',
       'overpayment_7_y', 'overpayment_8_y', 'overpayment_9_y',
       'overpayment_10_y', 'overpayment_11_y', 'overpayment_12_y',
       'overpayment_mess', 'overpayment_1', 'overpayment_2', 'overpayment_3',
       'overpayment_4', 'overpayment_5', 'overpayment_6', 'overpayment_7',
       'overpayment_8', 'overpayment_9', 'ove

In [64]:
revenue_avs = telecom_stats[['user_id', 'tariff_name', 'revenue_1', 'revenue_2',
       'revenue_3', 'revenue_4', 'revenue_5', 'revenue_6', 'revenue_7',
       'revenue_8', 'revenue_9', 'revenue_10', 'revenue_11', 'revenue_12']]

revenue_avs.columns = ['user_id', 'tariff_name', '1', '2','3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
# avs ОТ AVERANGE VARIANCE STD deviation

Посчитаем выручку отдельно по тарифам.

In [68]:
revenue_avs.loc['TOTAL_smart'] = revenue_avs.query('tariff_name in "smart"').sum()

In [66]:
revenue_avs.loc['TOTAL_ultra'] = revenue_avs.query('tariff_name in "ultra"').sum()

In [69]:
revenue_avs = revenue_avs.tail(2)
revenue_avs

,user_id,tariff_name,1,2,3,4,5,6,7,8,9,10,11,12
TOTAL_smart,436691,smartsmartsmartsmartsmartsmartsmartsmartsmarts...,22209.43,40460.54,75462.79,90851.22,144400.73,165142.95,206813.40,265138.01,272290.24,335482.67,350443.17,428117.07
TOTAL_ultra,188059,ultraultraultraultraultraultraultraultraultrau...,13650.00,27853.12,50235.21,74574.61,96712.35,126846.68,142801.61,167831.25,179770.75,199714.01,221189.94,233500.49


Удалим столбцы `user_id` и `tariff_name`, посчитаем показатели, а также транспонируем таблицу чтобы таблица была читабельной.

In [70]:
del revenue_avs['user_id']
del revenue_avs['tariff_name']

In [71]:
revenue_avs['mean'] = revenue_avs.mean(axis=1) # среднее по каждому из тарифов
revenue_avs['variance'] = np.var(revenue_avs.iloc[:, 0:12], axis=1, ddof=1) # дисперсия по каждому из тарифов
revenue_avs['std_deviation'] = np.std(revenue_avs.iloc[:, 0:12], axis=1, ddof=1) # станд.отклонение по каждому из тарифов

In [72]:
revenue_avs = revenue_avs.transpose() # переворачиваем таблицу
revenue_avs # выводим

,TOTAL_smart,TOTAL_ultra
1,22209.43,13650.00
2,40460.54,27853.12
3,75462.79,50235.21
4,90851.22,74574.61
5,144400.73,96712.35
6,165142.95,126846.68
7,206813.40,142801.61
8,265138.01,167831.25
9,272290.24,179770.75
10,335482.67,199714.01


Годовая выручка тарифа SMART в среднем превышает годовую выручку тарифа ULTRA на порядка 60 тысяч рублей, несмотря на то, что абонентская плата дороже. Не было ни одного месяца в году где тариф Ультра опережал Смарт.

Нельзя ориентироваться только на доходность, надо еще и смотреть на риски. Дисперсия тарифа Ультра меньше, а значит менее выгодней тариф, чем Смарт.

Стандратное отклонение изменяется в тех же величинах, что и исходные данные. Т.е. станд.отклоение по тарифу Смарт ~ составляет 125480.46 рублей.

В данном случае с показателем дисперсии работать не удобно, удобней ориентироваться по диаграмме размаха, она дает больше информации о скошенности.
Построим диаграмму `boxplot` с помощью библиотеки `plotly`.

In [73]:
revenue_hist = revenue_avs.iloc[:-3, :]
smart = revenue_hist['TOTAL_smart']
ultra = revenue_hist['TOTAL_ultra']

fig = go.Figure()
fig.add_trace(go.Box(x=smart, name='smart'))
fig.add_trace(go.Box(x=ultra, name='ultra'))

fig.show()

По данным о сумме выручки пользователей из разных каналов построены диаграммы размаха. Медианная выручка с тарифа смарт выше чем тарифа Ультра. Приблизительно в равные стороны скошен датасет, распределение больше похоже на нормальное.

Но пользователи выбирают тариф Смарт, выручка с него больше. Датасет также нормально распреден и скошен влево. По тарифу платят ощутимо больше, на этом факторе сказывается количество пользователей. Переплаты у них есть, по сравнению с пользователями тарифа Ультра. 

Если у тарифа Ультра снизить лимиты и соответственно абонентскую плату, то привлечём пользователей, которые будут активно пользоваться всеми услугами тарифнвых планов.

У дисперсии нет экономического смысла. Для этого мы далее должны перейти к показателею стандартного отклонения.
Для большинства распределений верно правило трёх стандартных отклонений, или правило трёх сигм. Оно гласит — практически все значения (около 99%) находятся в промежутке тех самых трехсигм в одну и другую стороны.

Это правило позволяет не только находить интервал, куда наверняка попадут практически все значения интересующей нас переменной, но и искать значения вне этого интервала. Их называют выбросами. На них полезно обратить внимание: само наличие выбросов может дать важную информацию об исследуемой переменной или указать на ошибки в её измерении.


####  Вывод по разделу

В ходе выполнения данного шага было описано поведение клиентов оператора, исходя из выборки. 
Мы посчитали сколько минут разговора, сколько сообщений и какой объём интернет-трафика требуется пользователям каждого тарифа в месяц. Посчитали среднее количество, дисперсию и стандартное отклонение, построили барчарты по показателям, а также описали распределения.

Из вышенаписаного следует сделать вывод, что компания достаточная новая, но новые пользователи активно растут. Пользователи активно пользуются интернетом, в среднем разговаривают немного и не стараются не привышать лимиты тарифов.

пользователи выбирают тариф Смарт, выручка с него больше. Датасет также нормально распреден и скошен влево. По тарифу платят ощутимо больше, на этом факторе сказывается количество пользователей. Переплаты у них есть, по сравнению с пользователями тарифа Ультра. 

Если у тарифа Ультра снизить лимиты и соответственно абонентскую плату, то привлечём пользователей, которые будут активно пользоваться всеми услугами тарифнвых планов.

Дисперсия может говорить о чувствительности выборки, дисперсия тарифа Смарт превышает тариф Ультра, это видно на всех этапах анализа. Очень хорошо экономическую выгоду видно на диаграмме размаха - ящик с усами.

На данном этапе мы могли бы предложить более гибкий тариф или возможность подключать/отключать необходимые опции самостоятельно.

---

### Шаг 4. Проверьте гипотезы

Проверьте гипотезы:

- средняя выручка пользователей тарифов «Ультра» и «Смарт» различается;
- средняя выручка пользователей из Москвы отличается от выручки пользователей из других регионов;
- Пороговое значение alpha задайте самостоятельно.

Поясните:
- как вы формулировали нулевую и альтернативную гипотезы;
- какой критерий использовали для проверки гипотез и почему.

Напомним, что у нас выручка пользователей хранится в таблице `telecom_stats`.

Чтобы понять, различается ли средняя выручка пользователей тарифов Смарт и Ультра, недостаточно просто сравнить числа за определённый период времени. Без проведения корректного теста нельзя считать отличия случайными или разницу достаточной, чтобы полагать выручку разной разными.

Одна из формулировок центральной предельной теоремы звучит так: **если в выборке достаточно наблюдений, выборочное распределение выборочного среднего из любой генеральной совокупности распределено нормально вокруг среднего этой генеральной совокупности.** 

«Любая генеральная совокупность» означает, что сама генеральная совокупность может быть распределена как угодно. 

Датасет из средних значений выборок всё равно будет нормально распределён вокруг среднего всей генеральной совокупности.

У нас телеком компания с двумя тарифами, и сейчас нам необходимо проверить гипотезу, что средняя выручка пользователей тарифов «Ультра» и «Смарт» различается.
Именно такой вывод мы сделали в третьем шаге при анализе выручки.

Начнём с формулировки нулевой гипотезы H₀. Т.к. она всегда со знаком равно, то будет звучать как, «средняя выручка пользователей тарифов «Ультра» и «Смарт» одинакова».
Исходя из H₀ формулируется альтернативная гипотеза H₁, и она уже будет звучать как "средняя выручка пользователей тарифов «Ультра» и «Смарт» различается".

Чтобы проверить гипотезу о равенстве среднего двух генеральных совокупностей по взятым из них выборкам, применим метод `scipy.stats.ttest_ind (array1, array2, equal_var) .`

Методу передают параметры:
- `array1, array2` — массивы, содержащие выборки;
- `equal_var` (от англ. equal variance, «равная дисперсия») — необязательный параметр, задающий считать ли равными дисперсии выборок. Передаётся как `equal_var = True` или `equal_var = False` (True — считать, False — не считать).

Приведены два ряда данных: выручка пользователей тарифов Смарт и Ультра за год.

In [74]:
telecom_stats.columns # напомним себе наименования столбцов датафрейма

Index(['user_id', 'age', 'churn_date', 'city', 'first_name', 'last_name',
       'reg_date', 'tariff_name', 'churn_month', 'messages_included',
       'mg_included', 'minutes_included', 'rub_monthly_fee', 'rub_per_gb',
       'rub_per_message', 'rub_per_minute', 'overpayment_1_x',
       'overpayment_2_x', 'overpayment_3_x', 'overpayment_4_x',
       'overpayment_5_x', 'overpayment_6_x', 'overpayment_7_x',
       'overpayment_8_x', 'overpayment_9_x', 'overpayment_10_x',
       'overpayment_11_x', 'overpayment_12_x', 'overpayment_calls',
       'overpayment_1_y', 'overpayment_2_y', 'overpayment_3_y',
       'overpayment_4_y', 'overpayment_5_y', 'overpayment_6_y',
       'overpayment_7_y', 'overpayment_8_y', 'overpayment_9_y',
       'overpayment_10_y', 'overpayment_11_y', 'overpayment_12_y',
       'overpayment_mess', 'overpayment_1', 'overpayment_2', 'overpayment_3',
       'overpayment_4', 'overpayment_5', 'overpayment_6', 'overpayment_7',
       'overpayment_8', 'overpayment_9', 'ove

In [75]:
smart_test = telecom_stats.query('tariff_name in "smart"')['revenue']
ultra_test = telecom_stats.query('tariff_name in "ultra"')['revenue']

alpha = .05 # критический уровень статистической значимости
            # если p-value окажется меньше него - отвергнем гипотезу

results = st.ttest_ind(
    smart_test, 
    ultra_test)

print('p-значение:', results.pvalue)

if (results.pvalue < alpha):
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

p-значение: 4.7794750815665496e-08
Отвергаем нулевую гипотезу


Полученное значение p-value говорит о том, что хотя средняя выручка пользователей двух тарифов неодинакова, такое различие получается не случайно. Это очень слишком большая вероятность, чтобы можно было сделать вывод о значимом различии между средней выручкой.

---

Посмотрим, что можно сказать о средней выручке пользователей из Москвы и отличается ли она от выручки пользователей из других регионов.

Для начала выведем в сводную таблицу по городам информацию по выручке за год. Для этого воспользуемся методом `pivot_table()`. В параметр `index` зададим города.

In [76]:
city_revenue = telecom_stats.pivot_table(index='city', values='revenue', aggfunc='sum')
city_revenue

,revenue
city,
Архангельск,14820.42
Астрахань,41171.03
Балашиха,38532.01
Барнаул,24658.53
Белгород,10913.59
...,...
Челябинск,85830.47
Череповец,27549.48
Чита,16864.60


Сформулируем альтернативную гипотезу (H₁) — «наблюдаемое значение больше заданного», или **"средняя выручка пользователей из Москвы больше выручки пользователей из других регионов"**. Вероятность попадания в критический интервал обычно весьма мала. Обнаружив в нём наблюдаемое на реальных данных значение, делаем вывод: оно достаточно далеко от предполагаемого, чтобы отвергнуть нулевую гипотезу.

Нам не важно, если наблюдаемое значение окажется слева от предполагаемого (пусть даже оно будет значительно меньше). Проверяем, не оказалось ли оно больше.
Если альтернативная гипотеза гласит «наблюдаемое значение меньше», то критический интервал будет слева. Тогда обнаружение выборочного среднего справа — это не повод отвергнуть нулевую гипотезу.

При одностороннем тесте вероятность, взятая для него за уровень значимости, приходится на одну сторону, а при двухстороннем — поровну делится на возможное отклонение вправо и влево.
В питоне нет специального метода для проведения одностороннего теста, есть только двухсторонний. Но если нужен односторонний тест, **выполним двухсторонний, а p-value просто поделим пополам**. Так мы получите односторонний уровень значимости отклонения наблюдаемого значения от предполагаемого. Это работает, потому что тест по умолчанию учитывает возможность отклонения в обе стороны. А поскольку двухсторонний тест симметричен, p-value будет ровно в два раза больше (чем если бы интересовало отклонение только в одну сторону).

Проверяя гипотезу, сравним выборочное среднее(средняя выручка по регионам) с предполагаемым значением(средняя выручка по Москве) — больше оно или нет. Когда альтернативная гипотеза гласит, что реальное значение меньше предполагаемого, а выборочное среднее — больше (неважно насколько), нет оснований отвергать нулевую гипотезу в пользу альтернативной. 

Если в этом случае наблюдаемое значение оказажется меньше предполагаемого, применим статистический тест.

In [77]:
revenue = city_revenue.query('city not in "Москва"')['revenue']

interested_value = city_revenue.query('city in "Москва"')['revenue'].mean()  # средняя выручка по Москве

alpha = .05 # критический уровень статистической значимости

results = st.ttest_1samp(
    revenue, 
    interested_value)

# тест односторонний: p-value будет в два раза меньше
print('средняя выручка за год по Москве: {:.2f}, средняя выручка за год в других регионах: {:.2f}'.format(interested_value, 
                                                                                                          revenue.mean()))
print('p-значение:', results.pvalue / 2)

if (results.pvalue / 2 < alpha) and (revenue.mean() < interested_value):
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

средняя выручка за год по Москве: 749631.59, средняя выручка за год в других регионах: 42424.81
p-значение: 5.754218439900876e-91
Отвергаем нулевую гипотезу


In [78]:
interested_value - revenue.mean()

707206.7850260417

Средняя выручка по Москве оказалась больше средней выручки по регионам , поэтому применять статистический тест не требуется.
Мы можем увидеть, что средняя выручка по регионам меньше выручки по Москве в среднем на 700 тысяч, перепроверять гипотезу не будем. Разница достаточно велика.

####  Вывод по разделу

Мы проверили гипотезы с помощью t-теста Стьюдента:
- средняя выручка пользователей тарифов «Ультра» и «Смарт» различается;
- средняя выручка пользователей из Москвы отличается от выручки пользователей из других регионов;
Пороговое значение alpha задавали в 5%.

Данные гипотезы подтвердились, в первом и втором случае - выручка тарифа Смарт в разы превышает выручку тарифа Ультра, поэтому p-value был равен почти 5.
Такая же ситуация обстоит с анализом выручки по Москве и регионам страны, в Москве сосредоточены клиенты и пользователи, которые приносят прибыли компании больше чем в остальных регионах, что составляет чуть более 700 тысяч рублей.

---

Нулевая гипотеза H₀ всегда со знаком равно, то звучала соответственно как, «средняя выручка пользователей тарифов «Ультра» и «Смарт» одинакова».
Исходя из H₀ формулировалась альтернативная гипотеза H₁ - "средняя выручка пользователей тарифов «Ультра» и «Смарт» различается".

Чтобы проверить гипотезу о равенстве среднего двух генеральных совокупностей по взятым из них выборкам, применяли метод `scipy.stats.ttest_ind (array1, array2, equal_var) .` В `array` применяли два ряда данных - по тарифу Смарт и тарифу Ультра.


Для второго задания мы сформулировали альтернативную гипотезу (H₁) — «наблюдаемое значение больше заданного», или **"средняя выручка пользователей из Москвы больше выручки пользователей из других регионов"**. Вероятность попадания в критический интервал была мала. При проверке можем сказать, что выручка в регионах достаточно далеко от выручки в Москве, чтобы отвергнуть нулевую гипотезу ("выручка в Москве и регионах равна между собой").

Для проверки гипотезы использовался односторонний тест, в питоне, как мы можем помнить, такого нет. Но мы выполнили двухсторонний, а p-value просто поделили пополам. 

### Шаг 5. Напишите общий вывод

В первом шаге мы подключили все интересующие нас библиотеки, которые потребовались в ходе работы, прочитали файлы, проверили на содержание, и поставили себе первые задачи для предобработки данных. "Грязную" работу выполняли постепенно в соответсвующих разделах.

Явных пропусков не обнаружено. **Только в таблице `users` в столбце `churn_date`(дата прекращения пользования тарифом) имеются у нас пропуски, а это значит, что тариф ещё действовал на момент выгрузки данных.** 

Таблица `tariffs` содержит всего две строки и носит информационный характер, в предобработке датафрейм не нуждался.

«Мегалайн» **всегда округляет вверх значения минут и мегабайтов.** Если пользователь проговорил всего 1 секунду, в тарифе засчитывается целая минута. Поэтому в разговорах мы округлили до единицы, а трафик мы не трогали, вероятно тарификация интернет-траффика не была, т.к интернет был отключен, а сигнал ушел.


Согласно заданию мы посчитали для каждого пользователя:
- количество израсходованных минут разговора по месяцам;
- количество отправленных сообщений по месяцам;
- объем израсходованного интернет-трафика по месяцам;

Помесячную выручку с каждого пользователя включили в соответствующую статистику по звонкам, мообщениям и интернет-трафику. Выручка считалась - как разница суммарного количества звонков/сообщений/интернет-трафика и бесплатного лимита; остаток умножить на значение из тарифного плана; прибавить абонентскую плату, соответствующую тарифному плану.

Мы посчитали сколько минут разговора, сколько сообщений и какой объём интернет-трафика требуется пользователям каждого тарифа в месяц. Посчитали среднее количество, дисперсию и стандартное отклонение, построили барчарты по показателям, а также описали распределения.

Компания достаточная новая, но новые пользователи активно растут. Пользователи активно пользуются интернетом, в среднем разговаривают немного и не стараются не привышать лимиты тарифов.

Пользователи выбирают тариф Смарт, выручка с него больше. Датасет также нормально распреден и скошен влево. По тарифу платят ощутимо больше, на этом факторе сказывается количество пользователей. Переплаты у них есть, по сравнению с пользователями тарифа Ультра. 

Если у тарифа Ультра снизить лимиты и соответственно абонентскую плату, то привлечём пользователей, которые будут активно пользоваться всеми услугами тарифнвых планов.

Дисперсия может говорить о чувствительности выборки, дисперсия тарифа Смарт превышает тариф Ультра, это видно на всех этапах анализа. Очень хорошо экономическую выгоду видно на диаграмме размаха - ящик с усами.

На данном этапе мы могли бы предложить более гибкий тариф или возможность подключать/отключать необходимые опции самостоятельно.

Нулевая гипотеза H₀ всегда со знаком равно, то звучала соответственно как, «средняя выручка пользователей тарифов «Ультра» и «Смарт» одинакова».
Исходя из H₀ формулировалась альтернативная гипотеза H₁ - "средняя выручка пользователей тарифов «Ультра» и «Смарт» различается".

Чтобы проверить гипотезу о равенстве среднего двух генеральных совокупностей по взятым из них выборкам, применяли метод `scipy.stats.ttest_ind (array1, array2, equal_var) .` В `array` применяли два ряда данных - по тарифу Смарт и тарифу Ультра.


Для второго задания в шаге 4 мы сформулировали альтернативную гипотезу (H₁) — «наблюдаемое значение больше заданного», или **"средняя выручка пользователей из Москвы больше выручки пользователей из других регионов"**. Вероятность попадания в критический интервал была мала. При проверке можем сказать, что выручка в регионах достаточно далеко от выручки в Москве, чтобы отвергнуть нулевую гипотезу ("выручка в Москве и регионах равна между собой"). Выручка в Москве на почти **700 тысяч больше** выручки по регионам.

Для проверки гипотезы использовался односторонний тест, в питоне, как мы можем помнить, такого нет. Но мы выполнили двухсторонний, а p-value просто поделили пополам. 

<div style="border:solid green 4px; padding: 20px">Работа отличная, было приятно читать, вопросов нет.</div>

## Чек-лист проекта

- [x] Описаны выявленные в данных проблемы
- [x] Подготовлены данные к анализу, предобработка данных
- [x] Построены графики для распределений
- [x] Даны выводы к графикам
- [x] Рассчитаны стандартное отклонение и дисперсия
- [x] Сформированы альтернативная и нулевая гипотезы
- [x] **Какие методы применяете для проверки гипотез?
- [x] **Интерпретируете ли результат проверки гипотезы?
- [x] Соблюдаете структуру проекта и поддерживаете аккуратность кода?
- [x] Сформулированы выводы по разделам
- [x] Сформулирован общий вывод
- [x] выбирать оптимальные метрики для описания данных;
- [x] определять типы гистограмм, необходимых для оценки дискретных и непрерывных величин;
- [x] делать выводы о данных по статистическим показателям;
- [x] понимать основы теории вероятностей;
- [x] определять и рассчитывать нормальное и биноминальное распределение;
- [x] строить и проверять гипотезы.